In [ ]:
import polars as pl
from pathlib import Path
import datetime as dt

import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent / "src"))

from conf import get_database_url

In [92]:
db_url = get_database_url(make_ipv6_in_bracket=True)

player_map_stats = pl.read_database_uri(
    """
    SELECT * FROM player_map_stats
    JOIN map_stats USING(map_stat_id)
    JOIN matches USING (match_id)
    JOIN events USING (event_id)
    """,
    db_url,
)
player_map_stats

event_id,match_id,map_stat_id,player_id,team_id,opening_kills_ct,opening_deaths_ct,multikills_ct,kast_ct,clutches_ct,kills_ct,headshot_kills_ct,assists_ct,flash_assists_ct,deaths_ct,traded_deaths_ct,adr_ct,swing_ct,rating_3_dot_0_ct,opening_kills_tr,opening_deaths_tr,multikills_tr,kast_tr,clutches_tr,kills_tr,headshot_kills_tr,assists_tr,flash_assists_tr,deaths_tr,traded_deaths_tr,adr_tr,swing_tr,rating_3_dot_0_tr,map_name,team_1_score,team_2_score,team_1_overtime_score,team_2_overtime_score,team_1_ct_score,team_1_tr_score,team_2_ct_score,team_2_tr_score,picked_by,starting_ct,match_date,team_1_id,team_2_id,team_1_map_score,team_2_map_score,team_winner_id,name,start_date,end_date,invite_date,vrs_date,vrs_weight,teams,total_prize_pool,player_share,location,event_type,has_top_50_teams
i32,i32,i32,i32,i32,i32,i32,i32,"decimal[38,10]",i32,i32,i32,i32,i32,i32,i32,"decimal[38,10]","decimal[38,10]","decimal[38,10]",i32,i32,i32,"decimal[38,10]",i32,i32,i32,i32,i32,i32,i32,"decimal[38,10]","decimal[38,10]","decimal[38,10]",str,i32,i32,i32,i32,i32,i32,i32,i32,str,str,"datetime[μs, UTC]",i32,i32,i32,i32,i32,str,"datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]",i32,i32,"decimal[38,10]","decimal[38,10]",str,str,bool
8895,2388750,215707,19617,11446,1,0,4,80.0000000000,1,17,5,2,1,6,0,151.4000000000,17.2300000000,3.0000000000,1,1,4,75.0000000000,0,12,6,5,1,6,2,107.9000000000,10.0900000000,1.9700000000,"""Ancient""",13,9,null,null,3,6,9,4,"""team_1""","""team_2""",2025-12-19 08:30:00 UTC,11446,4602,1,2,4602,"""United21 League Season 42""",null,null,null,null,null,null,null,null,null,null,false
8895,2388750,215707,22649,11446,1,5,2,60.0000000000,0,5,2,2,0,8,1,52.0000000000,-6.8300000000,0.5500000000,2,1,3,91.7000000000,1,12,4,3,0,6,3,95.0000000000,5.5900000000,1.5700000000,"""Ancient""",13,9,null,null,3,6,9,4,"""team_1""","""team_2""",2025-12-19 08:30:00 UTC,11446,4602,1,2,4602,"""United21 League Season 42""",null,null,null,null,null,null,null,null,null,null,false
8895,2388750,215707,21065,11446,0,0,1,80.0000000000,0,7,1,2,0,8,2,72.7000000000,-4.7600000000,0.7200000000,1,1,2,83.3000000000,0,9,1,3,0,5,1,70.1000000000,2.4500000000,1.2000000000,"""Ancient""",13,9,null,null,3,6,9,4,"""team_1""","""team_2""",2025-12-19 08:30:00 UTC,11446,4602,1,2,4602,"""United21 League Season 42""",null,null,null,null,null,null,null,null,null,null,false
8895,2388750,215707,21651,11446,0,3,0,60.0000000000,0,2,2,3,0,9,2,39.5000000000,-4.4000000000,0.4000000000,0,2,1,75.0000000000,0,9,5,2,0,8,1,81.9000000000,-1.4200000000,1.0500000000,"""Ancient""",13,9,null,null,3,6,9,4,"""team_1""","""team_2""",2025-12-19 08:30:00 UTC,11446,4602,1,2,4602,"""United21 League Season 42""",null,null,null,null,null,null,null,null,null,null,false
8895,2388750,215707,21650,11446,0,0,0,60.0000000000,0,4,2,0,0,10,3,41.7000000000,-5.1700000000,0.5400000000,1,2,1,83.3000000000,0,6,1,3,0,6,1,54.4000000000,1.1900000000,0.9800000000,"""Ancient""",13,9,null,null,3,6,9,4,"""team_1""","""team_2""",2025-12-19 08:30:00 UTC,11446,4602,1,2,4602,"""United21 League Season 42""",null,null,null,null,null,null,null,null,null,null,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
7359,2368495,167736,22997,10839,1,1,2,78.6000000000,0,10,6,5,1,11,2,73.4000000000,-8.3900000000,0.6500000000,1,2,2,73.3000000000,1,15,12,4,3,11,2,106.0000000000,12.9100000000,2.0300000000,"""Overpass""",16,13,1,4,6,6,6,6,"""leftover""","""team_2""",2023-12-06 09:00:00 UTC,4863,10839,2,1,4863,"""ESL Challenger League Season 4…",null,null,null,null,null,null,null,null,null,null,false
7359,2368495,167736,20258,10839,0,2,1,71.4000000000,0,6,3,4,0,10,2,69.0000000000,-4.4800000000,0.7200000000,1,2,4,66.7000000000,1,13,6,2,0,12,3,71.5000000000,-1.6500000000,1.1300000000,"""Overpass""",16,13,1,4,6,6,6,6,"""leftover""","""team_2""",2023-12-06 09:00:00 UTC,4863,10839,2,1,4863,"""ESL Challenger League Season 4…",null,null,null,null,null,

In [93]:
# List of stats to aggregate
stat_columns = [
    'kills', 'clutches', 'headshot', 'flash', 'assist', 'deaths', 
    'traded_deaths', 'adr', 'swing', 'rating_3_dot_0', 
    'opening_kills', 'opening_deaths', 'multikills', 'kast'
]

# Map column names from database to desired names
column_mapping = {
    'headshot_kills': 'headshot',
    'flash_assists': 'flash',
    'assists': 'assist'
}

# Create combined CT + TR columns for each stat
for stat in stat_columns:
    # Map the column name if needed
    if stat in column_mapping.values():
        # Find the original name
        original_stat = [k for k, v in column_mapping.items() if v == stat][0]
        ct_col = f"{original_stat}_ct"
        tr_col = f"{original_stat}_tr"
    else:
        ct_col = f"{stat}_ct"
        tr_col = f"{stat}_tr"
    
    # Sum CT and TR values
    player_map_stats = player_map_stats.with_columns(
        (pl.col(ct_col) + pl.col(tr_col)).alias(stat)
    )

player_map_stats.head()

event_id,match_id,map_stat_id,player_id,team_id,opening_kills_ct,opening_deaths_ct,multikills_ct,kast_ct,clutches_ct,kills_ct,headshot_kills_ct,assists_ct,flash_assists_ct,deaths_ct,traded_deaths_ct,adr_ct,swing_ct,rating_3_dot_0_ct,opening_kills_tr,opening_deaths_tr,multikills_tr,kast_tr,clutches_tr,kills_tr,headshot_kills_tr,assists_tr,flash_assists_tr,deaths_tr,traded_deaths_tr,adr_tr,swing_tr,rating_3_dot_0_tr,map_name,team_1_score,team_2_score,team_1_overtime_score,…,team_1_tr_score,team_2_ct_score,team_2_tr_score,picked_by,starting_ct,match_date,team_1_id,team_2_id,team_1_map_score,team_2_map_score,team_winner_id,name,start_date,end_date,invite_date,vrs_date,vrs_weight,teams,total_prize_pool,player_share,location,event_type,has_top_50_teams,kills,clutches,headshot,flash,assist,deaths,traded_deaths,adr,swing,rating_3_dot_0,opening_kills,opening_deaths,multikills,kast
i32,i32,i32,i32,i32,i32,i32,i32,"decimal[38,10]",i32,i32,i32,i32,i32,i32,i32,"decimal[38,10]","decimal[38,10]","decimal[38,10]",i32,i32,i32,"decimal[38,10]",i32,i32,i32,i32,i32,i32,i32,"decimal[38,10]","decimal[38,10]","decimal[38,10]",str,i32,i32,i32,…,i32,i32,i32,str,str,"datetime[μs, UTC]",i32,i32,i32,i32,i32,str,"datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]",i32,i32,"decimal[38,10]","decimal[38,10]",str,str,bool,i32,i32,i32,i32,i32,i32,i32,"decimal[38,10]","decimal[38,10]","decimal[38,10]",i32,i32,i32,"decimal[38,10]"
8895,2388750,215707,19617,11446,1,0,4,80.0000000000,1,17,5,2,1,6,0,151.4000000000,17.2300000000,3.0000000000,1,1,4,75.0000000000,0,12,6,5,1,6,2,107.9000000000,10.0900000000,1.9700000000,"""Ancient""",13,9,null,…,6,9,4,"""team_1""","""team_2""",2025-12-19 08:30:00 UTC,11446,4602,1,2,4602,"""United21 League Season 42""",null,null,null,null,null,null,null,null,null,null,false,29,1,11,2,7,12,2,259.3000000000,27.3200000000,4.9700000000,2,1,8,155.0000000000
8895,2388750,215707,22649,11446,1,5,2,60.0000000000,0,5,2,2,0,8,1,52.0000000000,-6.8300000000,0.5500000000,2,1,3,91.7000000000,1,12,4,3,0,6,3,95.0000000000,5.5900000000,1.5700000000,"""Ancient""",13,9,null,…,6,9,4,"""team_1""","""team_2""",2025-12-19 08:30:00 UTC,11446,4602,1,2,4602,"""United21 League Season 42""",null,null,null,null,null,null,null,null,null,null,false,17,1,6,0,5,14,4,147.0000000000,-1.2400000000,2.1200000000,3,6,5,151.7000000000
8895,2388750,215707,21065,11446,0,0,1,80.0000000000,0,7,1,2,0,8,2,72.7000000000,-4.7600000000,0.7200000000,1,1,2,83.3000000000,0,9,1,3,0,5,1,70.1000000000,2.4500000000,1.2000000000,"""Ancient""",13,9,null,…,6,9,4,"""team_1""","""team_2""",2025-12-19 08:30:00 UTC,11446,4602,1,2,4602,"""United21 League Season 42""",null,null,null,null,null,null,null,null,null,null,false,16,0,2,0,5,13,3,142.8000000000,-2.3100000000,1.9200000000,1,1,3,163.3000000000
8895,2388750,215707,21651,11446,0,3,0,60.0000000000,0,2,2,3,0,9,2,39.5000000000,-4.4000000000,0.4000000000,0,2,1,75.0000000000,0,9,5,2,0,8,1,81.9000000000,-1.4200000000,1.0500000000,"""Ancient""",13,9,null,…,6,9,4,"""team_1""","""team_2""",2025-12-19 08:30:00 UTC,11446,4602,1,2,4602,"""United21 League Season 42""",null,null,null,null,null,null,null,null,null,null,false,11,0,7,0,5,17,3,121.4000000000,-5.8200000000,1.4500000000,0,5,1,135.0000000000
8895,2388750,215707,21650,11446,0,0,0,60.0000000000,0,4,2,0,0,10,3,41.7000000000,-5.1700000000,0.5400000000,1,2,1,83.3000000000,0,6,1,3,0,6,1,54.4000000000,1.1900000000,0.9800000000,"""Ancient""",13,9,null,…,6,9,4,"""team_1""","""team_2""",2025-12-19 08:30:00 UTC,11446,4602,1,2,4602,"""United21 League Season 42""",null,null,null,null,null,null,null,null,null,null,false,10,0,3,0,3,16,4,96.1000000000,-3.9800000000,1.5200000000,1,2,1,143.3000000000


In [94]:
# Group by team_id and map_stat_id, then calculate aggregations
agg_expressions = []

for stat in stat_columns:
    agg_expressions.extend([
        pl.col(stat).mean().alias(f"{stat}_avg"),
        pl.col(stat).median().alias(f"{stat}_median"),
        pl.col(stat).quantile(0.25).alias(f"{stat}_p25"),
        pl.col(stat).quantile(0.75).alias(f"{stat}_p75"),
        pl.col(stat).std().alias(f"{stat}_std")
    ])

team_map_stats = player_map_stats.group_by(['team_id', 'map_stat_id']).agg(agg_expressions)

# Sort by team_id and map_stat_id for better readability
team_map_stats = team_map_stats.sort(['team_id', 'map_stat_id'])

print(f"Shape: {team_map_stats.shape}")
print(f"Columns: {len(team_map_stats.columns)}")
team_map_stats.head(10)

Shape: (86120, 72)
Columns: 72


team_id,map_stat_id,kills_avg,kills_median,kills_p25,kills_p75,kills_std,clutches_avg,clutches_median,clutches_p25,clutches_p75,clutches_std,headshot_avg,headshot_median,headshot_p25,headshot_p75,headshot_std,flash_avg,flash_median,flash_p25,flash_p75,flash_std,assist_avg,assist_median,assist_p25,assist_p75,assist_std,deaths_avg,deaths_median,deaths_p25,deaths_p75,deaths_std,traded_deaths_avg,traded_deaths_median,traded_deaths_p25,traded_deaths_p75,traded_deaths_std,adr_avg,adr_median,adr_p25,adr_p75,adr_std,swing_avg,swing_median,swing_p25,swing_p75,swing_std,rating_3_dot_0_avg,rating_3_dot_0_median,rating_3_dot_0_p25,rating_3_dot_0_p75,rating_3_dot_0_std,opening_kills_avg,opening_kills_median,opening_kills_p25,opening_kills_p75,opening_kills_std,opening_deaths_avg,opening_deaths_median,opening_deaths_p25,opening_deaths_p75,opening_deaths_std,multikills_avg,multikills_median,multikills_p25,multikills_p75,multikills_std,kast_avg,kast_median,kast_p25,kast_p75,kast_std
i32,i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2516,168654,17.4,17.0,14.0,18.0,4.722288,0.4,0.0,0.0,0.0,0.894427,9.4,9.0,8.0,10.0,2.302173,1.6,1.0,1.0,1.0,1.949359,7.8,7.0,5.0,8.0,4.969909,17.6,18.0,16.0,19.0,1.516575,4.4,4.0,4.0,5.0,1.140175,133.5,138.6,119.4,144.2,16.621973,4.12,2.89,1.76,4.83,3.757646,2.306,2.22,2.05,2.27,0.368008,3.4,3.0,3.0,4.0,1.140175,2.2,2.0,2.0,2.0,1.095445,3.8,4.0,3.0,4.0,1.48324,150.24,151.3,142.5,159.0,16.70892
2516,168663,24.6,26.0,22.0,28.0,3.974921,0.8,1.0,0.0,1.0,0.83666,12.2,12.0,12.0,14.0,4.024922,1.0,0.0,0.0,1.0,1.732051,7.6,7.0,5.0,8.0,3.286335,25.4,25.0,25.0,25.0,1.516575,4.4,4.0,4.0,5.0,1.140175,152.26,159.6,157.6,160.2,23.518248,-2.116,-3.59,-4.85,1.27,4.688302,2.036,2.04,2.0,2.32,0.360111,3.6,3.0,3.0,4.0,0.894427,3.4,3.0,1.0,6.0,3.04959,5.2,6.0,3.0,7.0,2.588436,137.6,142.1,136.9,147.3,17.356555
2516,168671,14.0,10.0,10.0,16.0,7.968689,0.2,0.0,0.0,0.0,0.447214,6.6,6.0,5.0,6.0,3.781534,1.6,1.0,0.0,2.0,2.073644,5.8,7.0,6.0,7.0,2.774887,16.8,18.0,14.0,18.0,2.683282,3.2,4.0,2.0,4.0,2.280351,131.48,130.8,92.8,164.4,62.631238,-1.894,-7.64,-7.74,-1.3,11.26415,1.874,1.57,1.21,1.69,1.07969,2.6,3.0,2.0,3.0,0.547723,2.0,2.0,1.0,2.0,1.870829,3.6,3.0,3.0,5.0,2.607681,130.88,130.3,122.7,131.0,10.704765
2516,168789,12.8,12.0,11.0,15.0,2.588436,0.2,0.0,0.0,0.0,0.447214,7.4,7.0,7.0,9.0,2.966479,0.4,0.0,0.0,1.0,0.547723,4.4,4.0,4.0,5.0,2.50998,16.6,16.0,16.0,17.0,0.894427,3.0,3.0,2.0,4.0,1.581139,132.3,133.6,132.1,136.6,12.583521,-5.526,-7.38,-9.22,-2.9,6.42649,1.718,1.71,1.64,1.98,0.364719,1.6,1.0,1.0,2.0,0.894427,2.6,3.0,2.0,3.0,1.140175,2.6,3.0,2.0,3.0,1.140175,128.92,130.6,122.3,138.9,12.521262
2516,168901,21.2,19.0,19.0,22.0,4.086563,0.4,0.0,0.0,1.0,0.547723,11.0,12.0,11.0,13.0,3.535534,1.6,1.0,0.0,3.0,1.81659,7.2,8.0,5.0,9.0,2.588436,21.2,20.0,20.0,22.0,1.788854,3.6,3.0,3.0,5.0,1.949359,155.1,150.9,133.7,159.7,30.720026,-1.232,-0.61,-3.17,-0.43,6.101403,2.162,1.95,1.92,2.24,0.412395,3.0,3.0,3.0,4.0,1.224745,3.0,3.0,2.0,4.0,1.0,5.8,4.0,3.0,8.0,3.563706,140.02,140.0,133.4,146.7,10.514609
2516,169489,10.0,9.0,8.0,11.0,3.162278,0.0,0.0,0.0,0.0,0.0,5.0,6.0,2.0,7.0,2.828427,0.2,0.0,0.0,0.0,0.447214,2.2,2.0,2.0,3.0,0.83666,13.6,14.0,13.0,14.0,0.547723,2.2,2.0,2.0,3.0,1.48324,123.6,137.7,98.5,145.5,33.763294,-10.566,-10.5,-12.04,-9.24,3.269049,1.468,1.43,1.37,1.74,0.33432,0.6,1.0,0.0,1.0,0.547723,2.8,3.0,2.0,4.0,1.30384,2.2,2.0,1.0,3.0,1.30384,122.7,126.7,106.7,141.7,26.315395
2516,170306,12.2,13.0,10.0,14.0,4.494441,0.2,0.0,0.0,0.0,0.447214,6.8,7.0,6.0,10.0,3.701351,1.0,1.0,0.0,1.0,1.224745,3.4,3.0,2.0,4.0,1.67332,16.2,16.0,16.0,16.0,1.095445,2.0,2.0,1.0,3.0,1.581139,124.24,114.4,98.7,160.5,48.312711,-5.042,-5.98,-6.25,-4.19,4.393981,1.67,1.44,1.42,2.26,0.604897,2.0,2.0,0.0,2.0,2.44949,2.2,2.0,1.0,2.0,1.643

## Create Match-Level Dataset

Now we'll create the final dataset where each row represents a match with:
- Team 1 and Team 2 IDs
- All statistics prefixed with `t1_` or `t2_`
- Winner column (0 = team 1 wins, 1 = team 2 wins)

In [95]:
# First, aggregate team statistics by map_stat_id (which is unique per map per match)
team_map_stats = player_map_stats.group_by(['team_id', 'map_stat_id']).agg(agg_expressions)

match_info = player_map_stats.select([
    'map_stat_id',
    'match_id',
    'team_1_id',
    'team_2_id',
    'team_winner_id',
    'map_name',
    'match_date',
    'event_id',
    'event_type',
    'vrs_weight',
    'has_top_50_teams'
]).unique()

print(f"Team stats shape: {team_map_stats.shape}")
print(f"Match info shape: {match_info.shape}")
team_map_stats.head()

Team stats shape: (86120, 72)
Match info shape: (43060, 11)


team_id,map_stat_id,kills_avg,kills_median,kills_p25,kills_p75,kills_std,clutches_avg,clutches_median,clutches_p25,clutches_p75,clutches_std,headshot_avg,headshot_median,headshot_p25,headshot_p75,headshot_std,flash_avg,flash_median,flash_p25,flash_p75,flash_std,assist_avg,assist_median,assist_p25,assist_p75,assist_std,deaths_avg,deaths_median,deaths_p25,deaths_p75,deaths_std,traded_deaths_avg,traded_deaths_median,traded_deaths_p25,traded_deaths_p75,traded_deaths_std,adr_avg,adr_median,adr_p25,adr_p75,adr_std,swing_avg,swing_median,swing_p25,swing_p75,swing_std,rating_3_dot_0_avg,rating_3_dot_0_median,rating_3_dot_0_p25,rating_3_dot_0_p75,rating_3_dot_0_std,opening_kills_avg,opening_kills_median,opening_kills_p25,opening_kills_p75,opening_kills_std,opening_deaths_avg,opening_deaths_median,opening_deaths_p25,opening_deaths_p75,opening_deaths_std,multikills_avg,multikills_median,multikills_p25,multikills_p75,multikills_std,kast_avg,kast_median,kast_p25,kast_p75,kast_std
i32,i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
11660,204995,17.4,19.0,14.0,19.0,5.029911,0.2,0.0,0.0,0.0,0.447214,8.8,10.0,4.0,12.0,4.604346,1.2,1.0,1.0,1.0,1.095445,4.6,4.0,3.0,5.0,2.073644,12.8,14.0,11.0,15.0,2.683282,3.4,3.0,2.0,5.0,2.073644,173.68,148.9,132.1,198.2,65.204195,2.414,3.99,-5.6,8.14,8.184228,2.516,2.6,1.73,2.7,0.887034,3.0,2.0,2.0,4.0,1.414214,1.2,1.0,1.0,2.0,0.83666,5.2,4.0,4.0,5.0,2.167948,152.8,155.6,152.8,161.1,13.160357
12471,169784,12.4,11.0,10.0,16.0,5.128353,0.6,0.0,0.0,1.0,0.894427,5.8,5.0,3.0,8.0,3.114482,0.2,0.0,0.0,0.0,0.447214,3.0,2.0,2.0,5.0,1.870829,15.2,15.0,15.0,15.0,1.095445,2.6,2.0,2.0,3.0,0.894427,131.98,107.1,102.3,175.5,44.205848,-4.27,-3.87,-5.59,-1.7,5.66336,1.82,1.92,1.43,2.21,0.58843,1.8,2.0,2.0,2.0,1.095445,2.4,2.0,2.0,3.0,1.140175,3.4,3.0,3.0,3.0,1.516575,122.74,127.7,108.3,136.1,23.045672
7020,164748,20.4,18.0,17.0,26.0,5.770615,0.8,1.0,0.0,1.0,0.83666,9.0,10.0,7.0,13.0,5.244044,2.2,1.0,1.0,3.0,2.387467,7.8,8.0,7.0,8.0,0.83666,18.4,20.0,17.0,20.0,2.880972,4.2,4.0,3.0,5.0,1.923538,149.58,141.8,140.1,183.1,37.146426,1.448,0.12,0.08,1.46,6.117685,2.222,2.14,1.88,2.41,0.42044,2.2,1.0,1.0,3.0,2.387467,3.6,5.0,1.0,6.0,2.880972,4.8,4.0,3.0,7.0,2.588436,148.94,151.9,151.9,151.9,15.528941
13219,197925,13.6,13.0,12.0,14.0,2.073644,1.0,1.0,1.0,1.0,0.707107,9.8,10.0,9.0,11.0,1.30384,1.0,1.0,0.0,1.0,1.224745,3.6,4.0,3.0,4.0,1.140175,8.8,9.0,8.0,10.0,1.30384,2.8,3.0,2.0,4.0,1.30384,165.76,159.2,151.6,192.5,51.534678,7.088,10.42,0.3,12.5,11.208547,3.016,3.24,2.02,3.76,0.980219,2.4,2.0,2.0,3.0,0.547723,0.8,1.0,0.0,1.0,0.83666,3.4,3.0,3.0,4.0,0.547723,168.34,175.0,158.3,191.7,27.905609
13180,215031,8.8,9.0,8.0,10.0,1.30384,0.2,0.0,0.0,0.0,0.447214,3.6,4.0,1.0,5.0,3.209361,0.8,0.0,0.0,0.0,1.788854,3.0,3.0,2.0,3.0,1.870829,13.6,14.0,13.0,15.0,1.67332,3.2,3.0,2.0,3.0,2.949576,122.16,131.9,110.0,136.7,20.903182,-13.724,-14.16,-16.13,-10.5,7.447861,1.354,1.19,1.12,1.39,0.375007,1.8,1.0,1.0,3.0,1.643168,1.2,1.0,1.0,2.0,0.83666,1.6,2.0,1.0,2.0,1.140175,138.32,150.0,125.0,158.3,30.418859


In [96]:
# Separate team 1 and team 2 statistics by matching team_id with team_1_id or team_2_id
# For team 1
team1_match_mapping = match_info.select(['map_stat_id', 'team_1_id'])
team1_stats = team_map_stats.join(
    team1_match_mapping,
    left_on=['map_stat_id', 'team_id'],
    right_on=['map_stat_id', 'team_1_id'],
    how='inner'
)

# For team 2
team2_match_mapping = match_info.select(['map_stat_id', 'team_2_id'])
team2_stats = team_map_stats.join(
    team2_match_mapping,
    left_on=['map_stat_id', 'team_id'],
    right_on=['map_stat_id', 'team_2_id'],
    how='inner'
)

print(f"Team 1 stats: {team1_stats.shape}")
print(f"Team 2 stats: {team2_stats.shape}")
print(f"Team 1 columns: {team1_stats.columns}")

Team 1 stats: (43060, 72)
Team 2 stats: (43060, 72)
Team 1 columns: ['team_id', 'map_stat_id', 'kills_avg', 'kills_median', 'kills_p25', 'kills_p75', 'kills_std', 'clutches_avg', 'clutches_median', 'clutches_p25', 'clutches_p75', 'clutches_std', 'headshot_avg', 'headshot_median', 'headshot_p25', 'headshot_p75', 'headshot_std', 'flash_avg', 'flash_median', 'flash_p25', 'flash_p75', 'flash_std', 'assist_avg', 'assist_median', 'assist_p25', 'assist_p75', 'assist_std', 'deaths_avg', 'deaths_median', 'deaths_p25', 'deaths_p75', 'deaths_std', 'traded_deaths_avg', 'traded_deaths_median', 'traded_deaths_p25', 'traded_deaths_p75', 'traded_deaths_std', 'adr_avg', 'adr_median', 'adr_p25', 'adr_p75', 'adr_std', 'swing_avg', 'swing_median', 'swing_p25', 'swing_p75', 'swing_std', 'rating_3_dot_0_avg', 'rating_3_dot_0_median', 'rating_3_dot_0_p25', 'rating_3_dot_0_p75', 'rating_3_dot_0_std', 'opening_kills_avg', 'opening_kills_median', 'opening_kills_p25', 'opening_kills_p75', 'opening_kills_std', 'o

In [97]:
# Rename team1 columns with t1_ prefix
t1_rename = {col: f't1_{col}' for col in team1_stats.columns if col not in ['map_stat_id', 'team_id']}
team1_stats = team1_stats.rename(t1_rename).rename({'team_id': 't1_id'})

# Rename team2 columns with t2_ prefix  
t2_rename = {col: f't2_{col}' for col in team2_stats.columns if col not in ['map_stat_id', 'team_id']}
team2_stats = team2_stats.rename(t2_rename).rename({'team_id': 't2_id'})

print("Team 1 columns sample:", team1_stats.columns[:10])
print("Team 2 columns sample:", team2_stats.columns[:10])

Team 1 columns sample: ['t1_id', 'map_stat_id', 't1_kills_avg', 't1_kills_median', 't1_kills_p25', 't1_kills_p75', 't1_kills_std', 't1_clutches_avg', 't1_clutches_median', 't1_clutches_p25']
Team 2 columns sample: ['t2_id', 'map_stat_id', 't2_kills_avg', 't2_kills_median', 't2_kills_p25', 't2_kills_p75', 't2_kills_std', 't2_clutches_avg', 't2_clutches_median', 't2_clutches_p25']


In [98]:
# Join team1 and team2 stats
match_dataset = team1_stats.join(
    team2_stats,
    on='map_stat_id',
    how='inner'
)

# Join with match info to get winner and event_type
match_dataset = match_dataset.join(
    match_info.select([
        'map_stat_id',
        'team_1_id',
        'team_2_id',
        'team_winner_id',
        'map_name',
        'match_date',
        'event_id',
        'event_type',
        'vrs_weight',
        'has_top_50_teams'
    ]),
    on='map_stat_id',
    how='inner'
)

# Create winner column: 0 if team_1 wins, 1 if team_2 wins
match_dataset = match_dataset.with_columns(
    (pl.col('team_winner_id') == pl.col('team_2_id')).cast(pl.Int32).alias('winner')
)

# Drop unnecessary columns (match_id, map_stat_id, player_id are already not present)
# Also drop team_winner_id since we have the winner column now
match_dataset = match_dataset.drop(['team_winner_id', 'team_1_id', 'team_2_id'])

print(f"\nFinal match dataset shape: {match_dataset.shape}")
print(f"Columns: {match_dataset.columns}")
print(f"\nWinner distribution:")
print(match_dataset.group_by('winner').len())
match_dataset.head()


Final match dataset shape: (43060, 150)
Columns: ['t1_id', 'map_stat_id', 't1_kills_avg', 't1_kills_median', 't1_kills_p25', 't1_kills_p75', 't1_kills_std', 't1_clutches_avg', 't1_clutches_median', 't1_clutches_p25', 't1_clutches_p75', 't1_clutches_std', 't1_headshot_avg', 't1_headshot_median', 't1_headshot_p25', 't1_headshot_p75', 't1_headshot_std', 't1_flash_avg', 't1_flash_median', 't1_flash_p25', 't1_flash_p75', 't1_flash_std', 't1_assist_avg', 't1_assist_median', 't1_assist_p25', 't1_assist_p75', 't1_assist_std', 't1_deaths_avg', 't1_deaths_median', 't1_deaths_p25', 't1_deaths_p75', 't1_deaths_std', 't1_traded_deaths_avg', 't1_traded_deaths_median', 't1_traded_deaths_p25', 't1_traded_deaths_p75', 't1_traded_deaths_std', 't1_adr_avg', 't1_adr_median', 't1_adr_p25', 't1_adr_p75', 't1_adr_std', 't1_swing_avg', 't1_swing_median', 't1_swing_p25', 't1_swing_p75', 't1_swing_std', 't1_rating_3_dot_0_avg', 't1_rating_3_dot_0_median', 't1_rating_3_dot_0_p25', 't1_rating_3_dot_0_p75', 't1_r

t1_id,map_stat_id,t1_kills_avg,t1_kills_median,t1_kills_p25,t1_kills_p75,t1_kills_std,t1_clutches_avg,t1_clutches_median,t1_clutches_p25,t1_clutches_p75,t1_clutches_std,t1_headshot_avg,t1_headshot_median,t1_headshot_p25,t1_headshot_p75,t1_headshot_std,t1_flash_avg,t1_flash_median,t1_flash_p25,t1_flash_p75,t1_flash_std,t1_assist_avg,t1_assist_median,t1_assist_p25,t1_assist_p75,t1_assist_std,t1_deaths_avg,t1_deaths_median,t1_deaths_p25,t1_deaths_p75,t1_deaths_std,t1_traded_deaths_avg,t1_traded_deaths_median,t1_traded_deaths_p25,t1_traded_deaths_p75,t1_traded_deaths_std,…,t2_swing_avg,t2_swing_median,t2_swing_p25,t2_swing_p75,t2_swing_std,t2_rating_3_dot_0_avg,t2_rating_3_dot_0_median,t2_rating_3_dot_0_p25,t2_rating_3_dot_0_p75,t2_rating_3_dot_0_std,t2_opening_kills_avg,t2_opening_kills_median,t2_opening_kills_p25,t2_opening_kills_p75,t2_opening_kills_std,t2_opening_deaths_avg,t2_opening_deaths_median,t2_opening_deaths_p25,t2_opening_deaths_p75,t2_opening_deaths_std,t2_multikills_avg,t2_multikills_median,t2_multikills_p25,t2_multikills_p75,t2_multikills_std,t2_kast_avg,t2_kast_median,t2_kast_p25,t2_kast_p75,t2_kast_std,map_name,match_date,event_id,event_type,vrs_weight,has_top_50_teams,winner
i32,i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,"datetime[μs, UTC]",i32,str,i32,bool,i32
2934,188061,16.2,17.0,14.0,18.0,5.167204,0.2,0.0,0.0,0.0,0.447214,4.8,6.0,4.0,6.0,1.788854,0.2,0.0,0.0,0.0,0.447214,4.2,4.0,4.0,6.0,2.04939,13.0,13.0,13.0,13.0,0.707107,2.6,2.0,1.0,3.0,2.073644,…,-3.07,-2.05,-5.24,-0.91,6.914387,1.794,1.53,1.36,1.65,0.735615,1.6,2.0,1.0,2.0,0.547723,2.6,2.0,1.0,2.0,3.130495,3.4,3.0,3.0,4.0,1.140175,118.86,108.3,102.7,119.4,29.730674,"""Mirage""",2024-11-14 06:10:00 UTC,7953,null,null,false,1
12634,172486,15.2,15.0,12.0,19.0,4.32435,0.6,1.0,0.0,1.0,0.547723,6.0,5.0,5.0,9.0,3.605551,0.6,0.0,0.0,1.0,0.894427,4.0,4.0,2.0,6.0,2.0,13.6,14.0,12.0,14.0,1.67332,2.0,1.0,1.0,3.0,1.414214,…,-2.672,-3.62,-6.8,0.6,7.917896,1.9,1.68,1.51,2.38,0.796963,2.2,2.0,1.0,2.0,2.280351,2.2,2.0,1.0,2.0,1.643168,3.0,3.0,2.0,4.0,1.581139,129.98,138.3,108.3,155.0,28.002982,"""Nuke""",2024-03-27 08:00:00 UTC,7686,null,null,false,0
12380,200123,14.0,15.0,13.0,16.0,5.147815,0.2,0.0,0.0,0.0,0.447214,6.4,6.0,3.0,9.0,3.577709,0.2,0.0,0.0,0.0,0.447214,5.2,6.0,3.0,7.0,2.588436,14.4,14.0,14.0,14.0,0.894427,2.6,3.0,1.0,4.0,1.516575,…,4.9,1.55,0.1,9.12,6.272436,2.416,2.28,2.17,2.59,0.520365,2.2,2.0,2.0,2.0,1.095445,2.0,1.0,1.0,3.0,2.0,3.2,4.0,2.0,4.0,1.095445,142.26,144.5,144.5,144.5,6.62216,"""Nuke""",2025-05-21 08:00:00 UTC,8477,"""Ranked""",25000,true,1
12586,207296,13.2,14.0,12.0,14.0,2.280351,0.4,0.0,0.0,1.0,0.547723,6.0,5.0,5.0,8.0,2.44949,0.8,0.0,0.0,1.0,1.30384,5.0,6.0,5.0,6.0,1.732051,9.2,9.0,9.0,10.0,0.83666,2.8,3.0,2.0,3.0,1.48324,…,-6.876,-5.68,-8.47,-5.3,2.316253,1.5,1.55,1.36,1.71,0.263723,1.6,2.0,1.0,2.0,0.547723,2.2,2.0,2.0,3.0,1.48324,2.6,3.0,2.0,3.0,1.140175,109.56,109.6,101.2,109.6,10.206273,"""Overpass""",2025-09-11 10:30:00 UTC,8660,"""Ranked""",20000,true,0
12144,202596,15.2,15.0,14.0,18.0,4.868265,0.2,0.0,0.0,0.0,0.447214,8.2,9.0,8.0,9.0,3.271085,1.0,0.0,0.0,2.0,1.414214,4.8,5.0,4.0,5.0,0.83666,14.2,14.0,12.0,16.0,2.280351,4.0,4.0,4.0,4.0,2.12132,…,-4.072,-3.72,-6.08,-2.37,5.141864,1.968,1.73,1.54,1.87,0.716778,3.0,4.0,2.0,4.0,2.0,1.4,1.0,1.0,2.0,0.547723,2.6,2.0,2.0,3.0,1.516575,143.0,143.3,135.0,146.7,9.481034,"""Inferno""",2025-07-09 18:05:00 UTC,8548,"""Ranked""",237500,true,0


In [99]:
# Get unique maps and sort them
unique_maps = match_dataset['map_name'].unique().sort()
print(f"Unique maps in dataset: {unique_maps.to_list()}")

map_to_encoding = {map_name: idx for idx, map_name in enumerate(unique_maps)}
print(f"\nMap encoding dictionary:")
for map_name, encoding in map_to_encoding.items():
    print(f"  {map_name}: {encoding}")

match_dataset = match_dataset.with_columns(
    pl.col('map_name').replace(map_to_encoding, return_dtype=pl.Int32).alias('map_encoding')
)

match_dataset = match_dataset.drop('map_name')

print(match_dataset.group_by('map_encoding').len().sort('map_encoding'))
print(f"Shape: {match_dataset.shape}")
match_dataset.head()

Unique maps in dataset: ['Ancient', 'Anubis', 'Cache', 'Dust2', 'Inferno', 'Mirage', 'Nuke', 'Overpass', 'Train', 'Vertigo']

Map encoding dictionary:
  Ancient: 0
  Anubis: 1
  Cache: 2
  Dust2: 3
  Inferno: 4
  Mirage: 5
  Nuke: 6
  Overpass: 7
  Train: 8
  Vertigo: 9
shape: (10, 2)
┌──────────────┬──────┐
│ map_encoding ┆ len  │
│ ---          ┆ ---  │
│ i32          ┆ u32  │
╞══════════════╪══════╡
│ 0            ┆ 8324 │
│ 1            ┆ 5262 │
│ 2            ┆ 3    │
│ 3            ┆ 4917 │
│ 4            ┆ 5220 │
│ 5            ┆ 7104 │
│ 6            ┆ 6234 │
│ 7            ┆ 2055 │
│ 8            ┆ 1886 │
│ 9            ┆ 2055 │
└──────────────┴──────┘
Shape: (43060, 150)


/var/folders/rp/lcxnvlln3qq7ym11pq6qf0w00000gn/T/ipykernel_2479/430455202.py:11: DeprecationWarning: the `return_dtype` parameter for `replace` is deprecated. Use `replace_strict` instead to set a return data type while replacing values.
(Deprecated in version 1.0.0)
  pl.col('map_name').replace(map_to_encoding, return_dtype=pl.Int32).alias('map_encoding')


t1_id,map_stat_id,t1_kills_avg,t1_kills_median,t1_kills_p25,t1_kills_p75,t1_kills_std,t1_clutches_avg,t1_clutches_median,t1_clutches_p25,t1_clutches_p75,t1_clutches_std,t1_headshot_avg,t1_headshot_median,t1_headshot_p25,t1_headshot_p75,t1_headshot_std,t1_flash_avg,t1_flash_median,t1_flash_p25,t1_flash_p75,t1_flash_std,t1_assist_avg,t1_assist_median,t1_assist_p25,t1_assist_p75,t1_assist_std,t1_deaths_avg,t1_deaths_median,t1_deaths_p25,t1_deaths_p75,t1_deaths_std,t1_traded_deaths_avg,t1_traded_deaths_median,t1_traded_deaths_p25,t1_traded_deaths_p75,t1_traded_deaths_std,…,t2_swing_avg,t2_swing_median,t2_swing_p25,t2_swing_p75,t2_swing_std,t2_rating_3_dot_0_avg,t2_rating_3_dot_0_median,t2_rating_3_dot_0_p25,t2_rating_3_dot_0_p75,t2_rating_3_dot_0_std,t2_opening_kills_avg,t2_opening_kills_median,t2_opening_kills_p25,t2_opening_kills_p75,t2_opening_kills_std,t2_opening_deaths_avg,t2_opening_deaths_median,t2_opening_deaths_p25,t2_opening_deaths_p75,t2_opening_deaths_std,t2_multikills_avg,t2_multikills_median,t2_multikills_p25,t2_multikills_p75,t2_multikills_std,t2_kast_avg,t2_kast_median,t2_kast_p25,t2_kast_p75,t2_kast_std,match_date,event_id,event_type,vrs_weight,has_top_50_teams,winner,map_encoding
i32,i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,"datetime[μs, UTC]",i32,str,i32,bool,i32,i32
2934,188061,16.2,17.0,14.0,18.0,5.167204,0.2,0.0,0.0,0.0,0.447214,4.8,6.0,4.0,6.0,1.788854,0.2,0.0,0.0,0.0,0.447214,4.2,4.0,4.0,6.0,2.04939,13.0,13.0,13.0,13.0,0.707107,2.6,2.0,1.0,3.0,2.073644,…,-3.07,-2.05,-5.24,-0.91,6.914387,1.794,1.53,1.36,1.65,0.735615,1.6,2.0,1.0,2.0,0.547723,2.6,2.0,1.0,2.0,3.130495,3.4,3.0,3.0,4.0,1.140175,118.86,108.3,102.7,119.4,29.730674,2024-11-14 06:10:00 UTC,7953,null,null,false,1,5
12634,172486,15.2,15.0,12.0,19.0,4.32435,0.6,1.0,0.0,1.0,0.547723,6.0,5.0,5.0,9.0,3.605551,0.6,0.0,0.0,1.0,0.894427,4.0,4.0,2.0,6.0,2.0,13.6,14.0,12.0,14.0,1.67332,2.0,1.0,1.0,3.0,1.414214,…,-2.672,-3.62,-6.8,0.6,7.917896,1.9,1.68,1.51,2.38,0.796963,2.2,2.0,1.0,2.0,2.280351,2.2,2.0,1.0,2.0,1.643168,3.0,3.0,2.0,4.0,1.581139,129.98,138.3,108.3,155.0,28.002982,2024-03-27 08:00:00 UTC,7686,null,null,false,0,6
12380,200123,14.0,15.0,13.0,16.0,5.147815,0.2,0.0,0.0,0.0,0.447214,6.4,6.0,3.0,9.0,3.577709,0.2,0.0,0.0,0.0,0.447214,5.2,6.0,3.0,7.0,2.588436,14.4,14.0,14.0,14.0,0.894427,2.6,3.0,1.0,4.0,1.516575,…,4.9,1.55,0.1,9.12,6.272436,2.416,2.28,2.17,2.59,0.520365,2.2,2.0,2.0,2.0,1.095445,2.0,1.0,1.0,3.0,2.0,3.2,4.0,2.0,4.0,1.095445,142.26,144.5,144.5,144.5,6.62216,2025-05-21 08:00:00 UTC,8477,"""Ranked""",25000,true,1,6
12586,207296,13.2,14.0,12.0,14.0,2.280351,0.4,0.0,0.0,1.0,0.547723,6.0,5.0,5.0,8.0,2.44949,0.8,0.0,0.0,1.0,1.30384,5.0,6.0,5.0,6.0,1.732051,9.2,9.0,9.0,10.0,0.83666,2.8,3.0,2.0,3.0,1.48324,…,-6.876,-5.68,-8.47,-5.3,2.316253,1.5,1.55,1.36,1.71,0.263723,1.6,2.0,1.0,2.0,0.547723,2.2,2.0,2.0,3.0,1.48324,2.6,3.0,2.0,3.0,1.140175,109.56,109.6,101.2,109.6,10.206273,2025-09-11 10:30:00 UTC,8660,"""Ranked""",20000,true,0,7
12144,202596,15.2,15.0,14.0,18.0,4.868265,0.2,0.0,0.0,0.0,0.447214,8.2,9.0,8.0,9.0,3.271085,1.0,0.0,0.0,2.0,1.414214,4.8,5.0,4.0,5.0,0.83666,14.2,14.0,12.0,16.0,2.280351,4.0,4.0,4.0,4.0,2.12132,…,-4.072,-3.72,-6.08,-2.37,5.141864,1.968,1.73,1.54,1.87,0.716778,3.0,4.0,2.0,4.0,2.0,1.4,1.0,1.0,2.0,0.547723,2.6,2.0,2.0,3.0,1.516575,143.0,143.3,135.0,146.7,9.481034,2025-07-09 18:05:00 UTC,8548,"""Ranked""",237500,true,0,4


## Event weighting

In [100]:
def weight_func(
    date: dt.datetime | None, 
    event_type: str | None, 
    vrs_weight: int | None, 
    has_top_50_teams: bool | None,
) -> float:
    if date is None:
        date = dt.datetime(2023, 12, 15)
    
    if date.tzinfo is not None:
        current_date = dt.datetime.now(tz=date.tzinfo)
    else:
        current_date = dt.datetime.now()
    
    days_old = (current_date - date).days
    recency_weight = 0.5 ** (days_old / 180)
    
    if vrs_weight is None:
        vrs_weight = 0
    
    vrs_scaled = 0.1 + (vrs_weight / 1000000) * 9.9
    
    if event_type is None:
        event_multiplier = 0.8
    elif event_type.lower() == 'ranked':
        event_multiplier = 1.5
    else:
        event_multiplier = 1.0
    
    if has_top_50_teams:
        top50_multiplier = 1.7
    else:
        top50_multiplier = 1.0
    
    total_weight = vrs_scaled * recency_weight * event_multiplier * top50_multiplier
    return total_weight

In [101]:
if match_dataset['match_date'].dtype != pl.Datetime:
    match_dataset = match_dataset.with_columns(
        pl.col('match_date').str.to_datetime()
    )

match_dataset = match_dataset.with_columns(
    pl.struct(['match_date', 'event_type', 'vrs_weight', 'has_top_50_teams']).map_elements(
        lambda x: weight_func(x['match_date'], x['event_type'], x['vrs_weight'], x['has_top_50_teams']),
        return_dtype=pl.Float64
    ).alias('event_weight')
)

match_dataset = match_dataset.drop(['event_type', 'vrs_weight', 'has_top_50_teams', 'event_id'])

print(f"\nEvent weight statistics:")
print(match_dataset['event_weight'].describe())

print(f"\nSample of matches with weights:")
print(match_dataset.select(['match_date', 'event_weight']).head(10))


Event weight statistics:
shape: (9, 2)
┌────────────┬───────────┐
│ statistic  ┆ value     │
│ ---        ┆ ---       │
│ str        ┆ f64       │
╞════════════╪═══════════╡
│ count      ┆ 43060.0   │
│ null_count ┆ 0.0       │
│ mean       ┆ 0.93995   │
│ std        ┆ 2.755421  │
│ min        ┆ 0.003455  │
│ 25%        ┆ 0.01457   │
│ 50%        ┆ 0.036893  │
│ 75%        ┆ 0.517032  │
│ max        ┆ 24.161658 │
└────────────┴───────────┘

Sample of matches with weights:
shape: (10, 2)
┌─────────────────────────┬──────────────┐
│ match_date              ┆ event_weight │
│ ---                     ┆ ---          │
│ datetime[μs, UTC]       ┆ f64          │
╞═════════════════════════╪══════════════╡
│ 2024-11-14 06:10:00 UTC ┆ 0.016497     │
│ 2024-03-27 08:00:00 UTC ┆ 0.006752     │
│ 2025-05-21 08:00:00 UTC ┆ 0.376899     │
│ 2025-09-11 10:30:00 UTC ┆ 0.49942      │
│ 2025-07-09 18:05:00 UTC ┆ 3.223123     │
│ 2024-06-16 13:05:00 UTC ┆ 0.009259     │
│ 2023-11-16 16:10:00 UTC ┆ 0.0040

In [102]:
print("\nTop 10 highest weighted matches:")
print(
    match_dataset.sort('event_weight', descending=True)
    .select(['map_stat_id', 'match_date', 'event_weight'])
    .head(10)
)


Top 10 highest weighted matches:
shape: (10, 3)
┌─────────────┬─────────────────────────┬──────────────┐
│ map_stat_id ┆ match_date              ┆ event_weight │
│ ---         ┆ ---                     ┆ ---          │
│ i32         ┆ datetime[μs, UTC]       ┆ f64          │
╞═════════════╪═════════════════════════╪══════════════╡
│ 215531      ┆ 2025-12-14 14:00:00 UTC ┆ 24.161658    │
│ 215532      ┆ 2025-12-14 14:00:00 UTC ┆ 24.161658    │
│ 215525      ┆ 2025-12-14 14:00:00 UTC ┆ 24.161658    │
│ 215528      ┆ 2025-12-14 14:00:00 UTC ┆ 24.161658    │
│ 215498      ┆ 2025-12-13 17:00:00 UTC ┆ 24.068795    │
│ 215499      ┆ 2025-12-13 17:00:00 UTC ┆ 24.068795    │
│ 215488      ┆ 2025-12-13 13:00:00 UTC ┆ 24.068795    │
│ 215496      ┆ 2025-12-13 17:00:00 UTC ┆ 24.068795    │
│ 215493      ┆ 2025-12-13 13:00:00 UTC ┆ 24.068795    │
│ 215455      ┆ 2025-12-12 16:00:00 UTC ┆ 23.976289    │
└─────────────┴─────────────────────────┴──────────────┘


In [103]:
output_path = Path('../data/match_dataset.csv')
output_path.parent.mkdir(parents=True, exist_ok=True)

if output_path.exists():
    output_path.unlink()
    print(f"Deleted existing file: {output_path}")

match_dataset.write_csv(output_path)

print(f"Exported to: {output_path}")
print(f"Rows: {match_dataset.shape[0]}")
print(f"Columns: {match_dataset.shape[1]}")
print(f"Size: {output_path.stat().st_size / 1024:.2f} KB")
print(f"\nColumns in exported dataset:")
print(match_dataset.columns)

Deleted existing file: ../data/match_dataset.csv
Exported to: ../data/match_dataset.csv
Rows: 43060
Columns: 147
Size: 47555.56 KB

Columns in exported dataset:
['t1_id', 'map_stat_id', 't1_kills_avg', 't1_kills_median', 't1_kills_p25', 't1_kills_p75', 't1_kills_std', 't1_clutches_avg', 't1_clutches_median', 't1_clutches_p25', 't1_clutches_p75', 't1_clutches_std', 't1_headshot_avg', 't1_headshot_median', 't1_headshot_p25', 't1_headshot_p75', 't1_headshot_std', 't1_flash_avg', 't1_flash_median', 't1_flash_p25', 't1_flash_p75', 't1_flash_std', 't1_assist_avg', 't1_assist_median', 't1_assist_p25', 't1_assist_p75', 't1_assist_std', 't1_deaths_avg', 't1_deaths_median', 't1_deaths_p25', 't1_deaths_p75', 't1_deaths_std', 't1_traded_deaths_avg', 't1_traded_deaths_median', 't1_traded_deaths_p25', 't1_traded_deaths_p75', 't1_traded_deaths_std', 't1_adr_avg', 't1_adr_median', 't1_adr_p25', 't1_adr_p75', 't1_adr_std', 't1_swing_avg', 't1_swing_median', 't1_swing_p25', 't1_swing_p75', 't1_swing_std